In [1]:
# -*- coding: utf-8 -*-

from rpy2 import robjects
from rpy2.robjects import FloatVector, pandas2ri
import os
from scipy.signal import argrelextrema
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.stats import skew, kurtosis
from statsmodels.tsa.stl._stl import STL
import warnings
from sklearn.preprocessing import MinMaxScaler

In [2]:
def read_data(path: str, nrows=None) -> pd.DataFrame:
    """
    Read the data file and return DataFrame.
    According to the provided file path, read the data file and return the corresponding DataFrame.
    :param path: The path to the data file.
    :return:  The DataFrame of the content of the data file.
    """
    data = pd.read_csv(path)
    label_exists = "label" in data["cols"].values

    all_points = data.shape[0]

    columns = data.columns

    if columns[0] == "date":
        n_points = data.iloc[:, 2].value_counts().max()
    else:
        n_points = data.iloc[:, 1].value_counts().max()

    is_univariate = n_points == all_points

    n_cols = all_points // n_points
    df = pd.DataFrame()

    cols_name = data["cols"].unique()

    if columns[0] == "date" and not is_univariate:
        df["date"] = data.iloc[:n_points, 0]
        col_data = {
            cols_name[j]: data.iloc[j * n_points: (j + 1) * n_points, 1].tolist()
            for j in range(n_cols)
        }
        df = pd.concat([df, pd.DataFrame(col_data)], axis=1)
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)

    elif columns[0] != "date" and not is_univariate:
        col_data = {
            cols_name[j]: data.iloc[j * n_points: (j + 1) * n_points, 0].tolist()
            for j in range(n_cols)
        }
        df = pd.concat([df, pd.DataFrame(col_data)], axis=1)

    elif columns[0] == "date" and is_univariate:
        df["date"] = data.iloc[:, 0]
        df[cols_name[0]] = data.iloc[:, 1]

        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)

    else:
        df[cols_name[0]] = data.iloc[:, 0]

    if label_exists:
        # Get the column name of the last column
        last_col_name = df.columns[-1]
        # Renaming the last column as "label"
        df.rename(columns={last_col_name: "label"}, inplace=True)

    if nrows is not None and isinstance(nrows, int) and df.shape[0] >= nrows:
        df = df.iloc[:nrows, :]

    return df

In [3]:
pandas2ri.activate()

# 定义 R 脚本并加载函数
r_script = '''
library(tidyverse)
library(Rcatch22)
library(forecast)
library(tsfeatures)

calculate_features <- function(dir_name, data){

  TSFEATURE_NAMES <- c("max_kl_shift",
                       "max_level_shift",
                       "max_var_shift",
                       "acf_features",
                       "arch_stat",
                       "crossing_points",
                       "entropy",
                       "flat_spots",
                       "holt_parameters",
                       "hurst",
                       "lumpiness",
                       "nonlinearity",
                       "pacf_features",
                       "stability",
                       "unitroot_kpss",
                       "unitroot_pp",
                       "firstmin_ac",
                       "firstzero_ac",
                       "trev_num",
                       "walker_propcross",
                       "std1st_der",
                       "histogram_mode",
                       "heterogeneity")

  all_features = tibble()
  catch_all_features = tibble()
  features = tibble()
  feature_type = "tsfeatures"

  i = 1
  seasonal.period = 1

  tryCatch(
    expr = {
      ts <- forecast:::msts(data, seasonal.periods = seasonal.period)

      if(feature_type == "tsfeatures"){ # Calculating tsfeatures
        feature <- tibble(file_name = dir_name)
        features <- tsfeatures:::tsfeatures(ts, c("mean","var"), scale = FALSE, na.rm = TRUE)
        features <- bind_cols(feature, features)
        for(f in TSFEATURE_NAMES){
          calculated_features <- tsfeatures:::tsfeatures(ts, features = f)

          if(sum(is.na(calculated_features)) > 0){ # if the calculated features contain missing values, then consider frequency as 1
            calculated_features <- tsfeatures:::tsfeatures(ts(ts, frequency = 1), features = f)

            if(sum(is.na(calculated_features)) > 0){ # Still if there are missing values, modify the parameters of the corresponding function
              if(f == "max_kl_shift" | f == "max_level_shift" | f == "max_var_shift")
                calculated_features <- tsfeatures:::tsfeatures(ts, features = f, width = 1)
              else{
                if(f == "arch_stat")
                  calculated_features <- tsfeatures:::tsfeatures(ts, features = f, lag = 1)
              }
            }
          }
          features <- bind_cols(features, calculated_features)
        }
        # Calculating stl_features
        tryCatch( 
          {
            seasonal_features <- tsfeatures:::tsfeatures(ts, "stl_features", s.window = 'periodic', robust = TRUE)
          },
          error = function(e) {
            tryCatch({
              seasonal_features <<- tsfeatures:::tsfeatures(ts, "stl_features")
            }, error = function(e) {
              seasonal_features <<- tsfeatures:::tsfeatures(ts(ts, frequency = 1), "stl_features") # Ignoring seasonality
            })
          }
        )

        catch_features <- catch22_all(ts)

        if(i == 1){
          catch_all_features <- as_tibble(matrix(NA, nrow = 1, ncol = 22)) # Creating a matrix to store the calculated features
          colnames(catch_all_features) <- catch_features$names
        }

        catch_all_features[i,] <- as_tibble(t(catch_features$values))
        i = i + 1

        features <- bind_cols(features, seasonal_features)
        lambdas <- tibble(lambdas = forecast:::BoxCox.lambda(ts))
        features <- bind_cols(features, lambdas)
      }
      all_features <- bind_rows(all_features, features)

    },
    error = function(err) {
      if (inherits(err, "error")) {
        cat("An error occurred: ", conditionMessage(err), "\n")
        cat("error file name:", dir_name, "\n")
      }
    }
  )
  catch_all_features <- na.omit(catch_all_features)
  all_features <- bind_cols(all_features, catch_all_features)

  return(all_features)
}
'''

# 执行 R 脚本以定义函数
robjects.r(r_script)

# 获取 R 的 calculate_features 函数
calculate_features = robjects.globalenv['calculate_features']

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [4]:
def compute_correlation(data):
    """
    Compute correlation from the last 22 columns of the input data.

    Parameters:
    data (pd.DataFrame or np.ndarray): The input data with at least 22 columns.

    Returns:
    float: The computed correlation value.
    """
    n_samples = data.shape[0]
    if n_samples == 1:
        return None

    # Ensure data is a NumPy array
    data = data.iloc[:, -22:]
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    corr_list = []

    # Compute absolute pairwise correlations
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            corr = abs(np.corrcoef(data_scaled[i], data_scaled[j])[0, 1])
            corr_list.append(corr)

    # Calculate statistics
    series_mean = np.mean(corr_list)
    series_var = np.var(corr_list)

    # Compute correlation
    correlation = 2 * (series_mean + 1 / (series_var + 2)) / 3

    return correlation

In [5]:


warnings.filterwarnings("ignore")

default_periods = [4, 7, 12, 24, 48, 52, 96, 144, 168, 336, 672, 1008, 1440]


def adjust_period(period_value):
    if abs(period_value - 4) <= 1:
        period_value = 4
    if abs(period_value - 7) <= 1:
        period_value = 7
    if abs(period_value - 12) <= 2:
        period_value = 12
    if abs(period_value - 24) <= 3:
        period_value = 24
    if abs(period_value - 48) <= 1 or (
        (48 - period_value) <= 4 and (48 - period_value) >= 0
    ):
        period_value = 48
    if abs(period_value - 52) <= 2:
        period_value = 52
    if abs(period_value - 96) <= 10:
        period_value = 96
    if abs(period_value - 144) <= 10:
        period_value = 144
    if abs(period_value - 168) <= 10:
        period_value = 168
    if abs(period_value - 336) <= 50:
        period_value = 336
    if abs(period_value - 672) <= 20:
        period_value = 672
    if abs(period_value - 720) <= 20:
        period_value = 720
    if abs(period_value - 1008) <= 100:
        period_value = 1008
    if abs(period_value - 1440) <= 200:
        period_value = 1440
    if abs(period_value - 8766) <= 500:
        period_value = 8766
    if abs(period_value - 10080) <= 500:
        period_value = 10080
    if abs(period_value - 21600) <= 2000:
        period_value = 21600
    if abs(period_value - 43200) <= 2000:
        period_value = 43200
    return period_value


def fftTransfer(timeseries, fmin=0.2):
    yf = abs(np.fft.fft(timeseries))  # Get the amplitude spectrum
    yfnormlize = yf / len(timeseries)  # Normalize
    yfhalf = yfnormlize[: len(timeseries) // 2] * 2  # Due to symmetry, only take half of the interval

    fwbest = yfhalf[
        argrelextrema(yfhalf, np.greater)
    ]  # Use the argrelextrema function to find the local maxima points in the frequency domain representation (points with amplitude greater than their adjacent points), these points represent the main frequency components in the time series; fwbest contains the amplitude values corresponding to these maxima points.
    xwbest = argrelextrema(yfhalf, np.greater)  # Find the indices of the local maxima points, representing their positions in the frequency domain representation.

    fwbest = fwbest[
        fwbest >= fmin
    ].copy()  # For maxima points with amplitude greater than or equal to fmin, filter them out and create a copy to ensure the original array is not modified. This step is based on the fmin parameter for amplitude filtering.

    return len(timeseries) / xwbest[0][: len(fwbest)], fwbest  # Return the period and the amplitude values of the frequency components that satisfy the conditions


def count_inversions(series):
    def merge_sort(arr):
        if len(arr) <= 1:
            return arr, 0

        mid = len(arr) // 2
        left, inversions_left = merge_sort(arr[:mid])
        right, inversions_right = merge_sort(arr[mid:])

        merged = []
        inversions = inversions_left + inversions_right

        i, j = 0, 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                j += 1
                inversions += len(left) - i

        merged.extend(left[i:])
        merged.extend(right[j:])

        return merged, inversions

    series_values = series.tolist()
    _, inversions_count = merge_sort(series_values)

    return inversions_count


def count_peaks_and_valleys(sequence):
    peaks = 0
    valleys = 0

    for i in range(1, len(sequence) - 1):
        if sequence[i] > sequence[i - 1] and sequence[i] > sequence[i + 1]:
            peaks += 1
        elif sequence[i] < sequence[i - 1] and sequence[i] < sequence[i + 1]:
            valleys += 1

    return peaks + valleys


def count_series(sequence, threshold):
    if len(sequence) == 0:
        return 0

    # Initialize the number of positive and negative subsequences
    positive_series = 0
    negative_series = 0

    # Flag to track the class of the current subsequence
    current_class = None

    for value in sequence:
        if value > threshold:
            # Current value belongs to the positive class
            if current_class == "negative":
                # If the previous subsequence is negative, increase the number of negative subsequences
                negative_series += 1
            current_class = "positive"
        else:
            # Current value belongs to the negative class
            if current_class == "positive":
                # If the previous subsequence is positive, increase the number of positive subsequences
                positive_series += 1
            current_class = "negative"

    # Finally, increase the number of subsequences based on the class of the last subsequence
    if current_class == "positive":
        positive_series += 1
    elif current_class == "negative":
        negative_series += 1

    return positive_series + negative_series


def extract_other_features(series_value):
    # Calculate skewness
    skewness = skew(series_value)

    # Calculate kurtosis  
    kurt = kurtosis(series_value)
    
    # Calculate relative standard deviation (RSD)
    rsd = abs((np.std(series_value) / np.mean(series_value)) * 100)
    
    # Calculate standard deviation of the first derivative
    std_of_first_derivative = np.std(np.diff(series_value))
    
    # Calculate the number of inversions
    inversions = count_inversions(series_value) / len(series_value)
    
    # Calculate the total number of peaks and valleys 
    turning_points = count_peaks_and_valleys(series_value) / len(series_value)
    
    # Calculate the number of subsequences
    series_in_series = count_series(series_value, np.median(series_value)) / len(
        series_value
)

    return [
        skewness,
        kurt,
        rsd,
        std_of_first_derivative,
        inversions,
        turning_points,
        series_in_series,
    ]



In [6]:
def feature_extract(path):
    index_columns = [
        "file_name",
        "length",
        "period_value1",
        "seasonal_strength1",
        "trend_strength1",
        "period_value2",
        "seasonal_strength2",
        "trend_strength2",
        "period_value3",
        "seasonal_strength3",
        "trend_strength3",
        "if_season",
        "if_trend",
        "ADF:p-value",
        "KPSS:p-value",
        "stability",
        "skewness",
        "kurt",
        "rsd",
        "std_of_first_derivative",
        "inversions",
        "turning_points",
        "series_in_series",
    ]
    result_frame = pd.DataFrame(columns=index_columns)

    file_name = path.split("/")[-1]
    file_name = [file_name]

    # Read data
    original_df = read_data(path)
    limited_length_df = original_df

    series_length = [original_df.shape[0]]

    for col in limited_length_df.columns:
        try:
            # ADF Test
            ADF_P_value = [adfuller(limited_length_df[col].values, autolag="AIC")[1]]

            # KPSS Test
            KPSS_P_value = [kpss(limited_length_df[col].values, regression="c")[1]]

            stability = [ADF_P_value[0] <= 0.05 or KPSS_P_value[0] >= 0.05]

        except:
            ADF_P_value = [None]
            KPSS_P_value = [None]
            stability = [None]

        series_value = limited_length_df[col]
        origin_series_value = original_df[col]
        series_value = pd.Series(series_value).astype("float")
        origin_series_value = pd.Series(origin_series_value).astype("float")
        other_features = extract_other_features(origin_series_value)
        periods, amplitude = fftTransfer(series_value, fmin=0)  # Fast Fourier Transform

        periods_list = []
        # Retain the periods corresponding to the largest amplitudes
        for index_j in range(len(amplitude)):
            periods_list.append(
                round(periods[amplitude.tolist().index(sorted(amplitude, reverse=True)[index_j])])
            )

        # Remove duplicates and periods less than 4
        final_periods1 = []
        for l1 in periods_list:
            l1 = adjust_period(l1)
            if l1 not in final_periods1 and l1 >= 4:
                final_periods1.append(l1)
        periods_num = min(len(final_periods1), 3)
        new_final_periods = final_periods1[:periods_num]
        new_final_periods = new_final_periods + default_periods

        final_periods = []
        for l1 in new_final_periods:
            if l1 not in final_periods and l1 >= 4:
                final_periods.append(l1)

        yuzhi = int(series_length[0] / 3)
        if yuzhi <= 12:
            yuzhi = 12

        season_dict = {}
        for index_period in range(len(final_periods)):
            period_value = final_periods[index_period]

            if period_value < yuzhi:
                res = STL(limited_length_df[col], period=period_value).fit()
                temp_df = pd.DataFrame()
                temp_df["original"] = limited_length_df[col]
                temp_df["trend"] = res.trend
                temp_df["seasonal"] = res.seasonal
                temp_df["resid"] = res.resid
                temp_df["detrend"] = temp_df["original"] - temp_df["trend"]
                temp_df["deseasonal"] = temp_df["original"] - temp_df["seasonal"]
                if temp_df["deseasonal"].var() == 0:
                    trend_strength = 0
                else:
                    trend_strength = max(
                        0, 1 - temp_df["resid"].var() / temp_df["deseasonal"].var()
                    )
                if temp_df["detrend"].var() == 0:
                    seasonal_strength = 0
                else:
                    seasonal_strength = max(
                        0, 1 - temp_df["resid"].var() / temp_df["detrend"].var()
                    )
                season_dict[seasonal_strength] = [
                    period_value,
                    seasonal_strength,
                    trend_strength,
                ]

        if len(season_dict) < 3:
            for i in range(3 - len(season_dict)):
                season_dict[0.1 * (i + 1)] = [0, -1, -1]

        season_dict = sorted(season_dict.items(), key=lambda x: x[0], reverse=True)

        result_list = []

        for num, (key, value) in enumerate(season_dict):
            if num == 0:
                max_seasonal_strength = value[1]
                max_trend_strength = value[2]
            if num <= 2:
                result_list = result_list + value

        if_seasonal = [max_seasonal_strength >= 0.9]
        if_trend = [max_trend_strength >= 0.85]
        result_list = (
            file_name
            + series_length
            + result_list
            + if_seasonal
            + if_trend
            + ADF_P_value
            + KPSS_P_value
            + stability
            + other_features
        )

        result_frame.loc[len(result_frame.index)] = result_list
    return result_frame


In [7]:
def process_file(file_path):
    """
    Process a single CSV file, call R's calculate_features function, and return a DataFrame of features.
    """

    data = read_data(file_path)
    
    col_names = data.columns.tolist()
    num_cols = data.shape[1]
    
    # Check if "label" column exists
    if "label" in col_names:
        num_cols -= 1
    
    # Initialize a list to store the features for each column 
    file_results = []
    
    # Iterate over each column to process
    for i in range(num_cols):
        col_name = col_names[i]
        col_data = data.iloc[:, i].dropna().tolist()  # Remove missing values and convert to list
    
        # Convert data to R numeric vector
        r_data = FloatVector(col_data)
    
        
            # Call R function and get return value
        features = calculate_features(file_path, r_data)
    
            # Convert R DataFrame to Pandas DataFrame 
        df_features = pandas2ri.rpy2py(features)
    
            # Add results to list
        file_results.append(df_features)
    
    # Return merged results
    if file_results:
        return pd.concat(file_results, ignore_index=True)
    else:
        return None


def process_path(file_path):
    """
    Based on the provided file_path type (file or directory), process CSV files and return a DataFrame of all features.
    """
    all_results = []

    if os.path.isfile(file_path) and file_path.lower().endswith('.csv'):
        # Process a single CSV file
        print(f"Detected a single CSV file: {file_path}")
        result=pd.concat([feature_extract(file_path).loc[:, ["seasonal_strength1","trend_strength1","ADF:p-value"]],process_file(file_path)], axis=1)  
        if result is not None:
            all_results.append(result)
    elif os.path.isdir(file_path):
        # Process all CSV files in the directory
        print(f"Detected a directory: {file_path}") 
        file_names = [f for f in os.listdir(file_path) if
                      os.path.isfile(os.path.join(file_path, f)) and f.lower().endswith('.csv')]
    
        if not file_names:
            print(f"No CSV files found in the directory: {file_path}")
            return pd.DataFrame()  # Return empty DataFrame
    
        for file_name in file_names:
            full_path = os.path.join(file_path, file_name)
            result=pd.concat([process_file(full_path),feature_extract(full_path).drop(columns=['file_name'])], axis=1)
            if result is not None:
                all_results.append(result)
    else:
        print(f"The provided path is neither a CSV file nor a directory: {file_path}")
        return pd.DataFrame()  # Return empty DataFrame
    
    # If needed, merge all results into a single DataFrame   
    result_list = []
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True).groupby('file_name')   
        for file_name, group in combined_results:
            correlation=compute_correlation(group)
            
            mean_result = group.drop(columns=['file_name']).mean()
            mean_result['file_name'] = file_name
            mean_result['correlation'] = correlation  
            result_list.append(mean_result)
        result_df = pd.DataFrame()    
        for df in result_list:
            result_df = pd.concat([result_df, pd.DataFrame(df).transpose()], axis=0)
    
        result_df = result_df[['file_name'] + [col for col in result_df.columns if col != 'file_name']] 
        print("All features have been calculated.")
        result=result_df.loc[:, ["file_name","correlation","SB_TransitionMatrix_3ac_sumdiagcov","DN_OutlierInclude_p_001_mdrmd","seasonal_strength1","trend_strength1","ADF:p-value"]]
        result.columns = ["file_name","Correlation","Transition","Shifting","Seasonality","Trend","Stationarity"]
        
        result['Shifting'] = result['Shifting'].abs()
        result['Correlation'] = result['Correlation'] * 3
        return result
    else:
        print("No features were calculated.")
        return pd.DataFrame()  # Return empty DataFrame

In [8]:
if __name__ == "__main__":
    # You can change this path to the desired file or directory path
    file_path = "univariate_datasets"  # Can be a single CSV file or a directory containing multiple CSV files
    # Call the function to process the path

    results = process_path(file_path)
    
    # Check results and save to CSV (optional)
    if not results.empty:
        output_path = 'features_output2.csv'
        results.to_csv(output_path, index=False)
        print(f"Features have been saved to {output_path}")
    else:
        print("No features to save.")

Detected a directory: RRRRR
All features have been calculated.
Features have been saved to features_output2.csv
